In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import string

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import re
import keras


In [2]:
vocab = (
    list(string.ascii_letters)
    + list(string.digits)
    + list(string.punctuation)
    + [" ", "[UNK]", "\u2705", "\u274C"]
)

In [3]:
def tokenize(vocab: list[str], input: str) -> torch.Tensor:
    idxs = []
    for char in input:
        try:
            idx = vocab.index(char)
        except ValueError:
            idx = vocab.index("[UNK]")
        idxs.append(idx)
    return torch.as_tensor(idxs)


def detokenize(vocab: list[str], input: torch.Tensor) -> str:
    output = ""
    for idx in input:
        output += vocab[idx]
    return output


class MovieReviewDataset(Dataset):
    def __init__(self, review_dir: str, vocab: list[str]) -> None:
        self.dir = review_dir
        self.vocab = vocab
        self.files = [item for item in os.listdir(review_dir)]

    def __len__(self) -> int:
        return len(self.files)

    def __getitem__(self, index: int) -> str:
        with open(self.dir + self.files[index], "r") as file:
            string = file.read()
            # Remove extraneous HTML tags
            string = re.sub(r"<[^>]*>", "", string)
            # Add beginning and ending indicator characters
            # These are 2705: ✅, 274C: ❌
            string = "\u2705" + string + "\u274C"
        tokens = tokenize(self.vocab, string)
        return tokens[:-1], tokens[1:]

In [4]:
def collate_fn(batch):
    inputs = [x[0] for x in batch]
    outputs = [x[1] for x in batch]
    return nn.utils.rnn.pad_sequence(
        inputs, padding_value=999
    ), nn.utils.rnn.pad_sequence(outputs, padding_value=999)


dataset = MovieReviewDataset("./data/movie_reviews/train/unsup/", vocab=vocab)
dataloader = DataLoader(
    dataset, batch_size=64, shuffle=True, collate_fn=collate_fn
)

In [5]:
input = keras.Input(shape=(None,))

embed = keras.layers.Embedding(len(vocab), len(vocab))
mask = keras.layers.Masking(mask_value=999)
lstm = keras.layers.LSTM(256, return_sequences=True)
dense = keras.layers.Dense(len(vocab))

output = dense(lstm(mask(embed(input))))

model = keras.Model(inputs=input, outputs=output)

/Users/erikdains/Programming/PythonBlogProjects/venv/lib/python3.11/site-packages/keras/src/backend/torch/linalg.py:53: UserWarning: The operator 'aten::linalg_qr.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch.linalg.qr(x, mode=mode)


In [6]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 98)  │      9,604 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None, 98)  │          0 │ embedding[0][0]   │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking (Masking)   │ (None, None, 98)  │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any (Any)           │ (None, None)      │          0 │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, None, 256) │    363,520 │ masking[0][0],    │
│                     │                   │            │ any[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 98)  │     25,186 │ lstm[0][0]        │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 398,310 (1.52 MB)

 Trainable params: 398,310 (1.52 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=keras.optimizers.AdamW(), metrics=["accuracy"])

In [8]:
history = model.fit(dataloader, epochs=10)

Epoch 1/10
 10/782 ━━━━━━━━━━━━━━━━━━━━ 2:25:36 11s/step - accuracy: 0.0302 - loss: 1.1945